# Import Library

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import to_networkx, subgraph, k_hop_subgraph
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, GINConv
from torch_geometric.data import Data
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import random

In [ ]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [ ]:
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load Cora Dataset

In [ ]:
dataset = Planetoid(root="../data/Cora", name="Cora")
data = dataset[0]

In [ ]:
node_of_interest = random.randint(0, data.num_nodes - 1)
print("Node of Interest:", node_of_interest)

# extract 2-hop subgraph
subset_nodes, sub_edge_index, _, _ = k_hop_subgraph(
    node_of_interest, 
    num_hops=2, 
    edge_index=data.edge_index, 
    relabel_nodes=True
)

sub_x = data.x[subset_nodes]
sub_y = data.y[subset_nodes]

print("Subgraph Nodes:", sub_x.size(0))

In [ ]:
#Jumlah Node
data.num_nodes

In [ ]:
#jumlah edge
data.num_edges

In [ ]:
#dimensi fitur
data.num_features

In [ ]:
#jumlah keasl
dataset.num_classes

In [ ]:
#nama kelas
data.y.unique()

In [ ]:
#nodes belong to classes
data.y.bincount()

In [ ]:
class_counts = data.y.bincount()
for class_idx in range(len(class_counts)):
    print(f"Kelas {class_idx}: {class_counts[class_idx].item()} nodes")

In [ ]:
#train nodes
data.train_mask.sum().item()

In [ ]:
print(f"\nTrain ratio: {data.train_mask.sum().item()/data.num_nodes:.2%}")

In [ ]:
#validation nodes
data.val_mask.sum().item()

In [ ]:
print(f"Validation ratio: {data.val_mask.sum().item()/data.num_nodes:.2%}")

In [ ]:
#test nodes
data.test_mask.sum().item()

In [ ]:
print(f"Test ratio: {data.test_mask.sum().item()/data.num_nodes:.2%}")

In [ ]:
data.edge_index.shape

In [ ]:
data.edge_index[:, :5]

In [ ]:
data.x[0, :10]

# SAMPLING & SUBGRAPH EXTRACTION (2-3 HOPS)

In [ ]:
node_of_interest = random.randint(0, data.num_nodes - 1)

In [ ]:
print(f"Node of Interest: {node_of_interest} (Class: {data.y[node_of_interest].item()})")

In [ ]:
subset_nodes, sub_edge_index, mapping, _ = k_hop_subgraph(
    node_of_interest, 
    num_hops=2,  # Bisa diganti 3 untuk 3-hops
    edge_index=data.edge_index, 
    relabel_nodes=True,
    num_nodes=data.num_nodes
)

sub_x = data.x[subset_nodes]
sub_y = data.y[subset_nodes]
sub_train_mask = data.train_mask[subset_nodes]
sub_val_mask = data.val_mask[subset_nodes]

print(f"Subgraph Nodes: {sub_x.size(0)}")
print(f"Subgraph Edges: {sub_edge_index.size(1)}")
print(f"Classes in Subgraph: {torch.unique(sub_y).tolist()}")

# VISUALIZE SUBGRAPH

In [ ]:
subgraph_data = Data(x=sub_x, edge_index=sub_edge_index)
G = to_networkx(subgraph_data, to_undirected=True)

plt.figure(figsize=(14, 10))
cmap = plt.cm.tab10
node_colors = [cmap(y) for y in sub_y.numpy()]
pos = nx.spring_layout(G, seed=42, k=1.5)

# Find center node index in subgraph
center_idx = (subset_nodes == node_of_interest).nonzero(as_tuple=True)[0].item()

# Draw nodes and edges
nx.draw_networkx_nodes(G, pos, node_size=300, node_color=node_colors, 
                       alpha=0.8, linewidths=0.5, edgecolors='black')
nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5, edge_color='gray')

# Highlight center node
nx.draw_networkx_nodes(G, pos, nodelist=[center_idx], node_size=600, 
                       node_color='red', edgecolors='black', linewidths=3)

labels = {center_idx: f'Center\n{node_of_interest}'}
nx.draw_networkx_labels(G, pos, labels, font_size=10, font_weight='bold')

plt.title(f"2-hop Subgraph of Node {node_of_interest} (Class {data.y[node_of_interest].item()})", 
          fontsize=16, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
degrees = np.zeros(data.num_nodes)
for i in range(data.num_nodes):
    degrees[i] = (data.edge_index[0] == i).sum().item()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of degree distribution
axes[0].hist(degrees, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Degree')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Degree Distribution Histogram')
axes[0].grid(True, alpha=0.3)

# Log-log plot for power law check
unique_degrees, degree_counts = np.unique(degrees, return_counts=True)
axes[1].loglog(unique_degrees, degree_counts, 'bo', alpha=0.7)
axes[1].set_xlabel('Degree (log)')
axes[1].set_ylabel('Frequency (log)')
axes[1].set_title('Degree Distribution (Log-Log Scale)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
print(f"• Average degree: {degrees.mean():.2f}")
print(f"• Maximum degree: {degrees.max():.0f}")
print(f"• Minimum degree: {degrees.min():.0f}")

# PRA-PROSES (Message Passing Layer)

$h_u^{(k)} = \sum W_{\text{msg}} \cdot h_u^{(k-1)} + W_{\text{self}} \cdot h_v^{(k-1)}$

 di mana,
 1. **k** adalah urutan lapisan GNN.
 2. **Wmsg** adalah bobot yang diakses bersama dengan neighborhood nodes.
 3. **Wself** adalah bobot milik node asal (v).

In [ ]:
class CustomMessagePassing(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')
        self.W_msg = nn.Linear(in_channels, out_channels, bias=False)
        self.W_self = nn.Linear(in_channels, out_channels, bias=False)
        
    def forward(self, x, edge_index):
        return self.propagate(edge_index, x=x)
    
    def message(self, x_j):
        return self.W_msg(x_j)
    
    def update(self, aggr_out, x):
        return aggr_out + self.W_self(x)


In [ ]:
# # TEST MESSAGE PASSING

# test_mp = CustomMessagePassing(in_channels=16, out_channels=32)
# test_mp

In [ ]:
# test_mp.W_msg.weight.shape

In [ ]:
# test_mp.W_self.weight.shape

# Intra-layer GNN Block

$$
\mathbf{h}_v^{(k)} =
\text{ACT}\!\left(
    \text{Dropout}\!\left(
        \text{BatchNorm}\!\left(
            \mathbf{W}^{(k-1)}\,\mathbf{h}_v^{(k-1)} + \mathbf{b}^{(k-1)}
        \right)
    \right)
\right)
+ \mathbf{h}_v^{(k-1)}
$$


di mana:

1. **ACT** adalah fungsi aktivasi yang Anda pilih.
2. **BatchNorm** adalah `1D-Batch Normalization`.


## DEFINE SINGLE GNN LAYER 

In [ ]:
class GNNLayer(nn.Module):
    def __init__(self, in_channels, out_channels, dropout=0.5):
        super().__init__()
        
        # Message passing (pre-process)
        self.message_passing = CustomMessagePassing(in_channels, out_channels)
        
        # Post-processing
        self.linear = nn.Linear(out_channels, out_channels)
        self.batch_norm = nn.BatchNorm1d(out_channels)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU()
        
        # Residual connection
        if in_channels != out_channels:
            self.residual = nn.Linear(in_channels, out_channels)
        else:
            self.residual = nn.Identity()
    
    def forward(self, x, edge_index):
        x_input = x
        
        # 1. Message passing
        x = self.message_passing(x, edge_index)
        
        # 2. Linear transformation
        x = self.linear(x)
        
        # 3. Batch Normalization
        x = self.batch_norm(x)
        
        # 4. Dropout
        x = self.dropout(x)
        
        # 5. Activation
        x = self.activation(x)
        
        # 6. Residual connection
        x = x + self.residual(x_input)
        
        return x

## Test GNN Layer

In [ ]:
# test_layer = GNNLayer(in_channels=16, out_channels=32)
# test_layer

In [ ]:
# test_layer.batch_norm

In [ ]:
# test_layer.dropout.p

In [ ]:
# test_layer.activation

# Full Model with k>1 Layers

In [ ]:
class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes, 
                 num_layers=3, dropout=0.5):
        super().__init__()
        
        assert num_layers > 1, "num_layers must be > 1"
        
        # Input projection
        self.input_proj = nn.Linear(in_channels, hidden_channels)
        
        # GNN Layers (k > 1)
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            if i == 0:
                self.layers.append(GNNLayer(hidden_channels, hidden_channels, dropout))
            else:
                self.layers.append(GNNLayer(hidden_channels, hidden_channels, dropout))
        
        # Output layer
        self.output_layer = nn.Linear(hidden_channels, out_channels)
        
        # Classifier
        self.classifier = nn.Linear(out_channels, num_classes)
        
    def forward(self, x, edge_index):
        # Input projection
        x = self.input_proj(x)
        x = F.relu(x)
        
        # GNN layers
        for layer in self.layers:
            x = layer(x, edge_index)
        
        # Output
        x = self.output_layer(x)
        x = F.relu(x)
        x = self.classifier(x)
        
        return F.log_softmax(x, dim=1)

# Model Structure

In [ ]:
model = GNNModel(
    in_channels=data.num_features,
    hidden_channels=128,
    out_channels=64,
    num_classes=dataset.num_classes,
    num_layers=3,
    dropout=0.6
)

In [ ]:
total_params = sum(p.numel() for p in model.parameters())

In [ ]:
total_params

In [ ]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
trainable_params

In [ ]:
for name, param in model.named_parameters():
    print(f"{name:50s} | Shape: {str(list(param.shape)):20s} | Params: {param.numel():,}")

# TRAINING SETUP

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
model = model.to(device)
data = data.to(device)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.NLLLoss()

In [ ]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=10
)

# TRAINING LOOP

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(mask):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out[mask].argmax(dim=1)
        correct = (pred == data.y[mask]).sum().item()
        accuracy = correct / mask.sum().item()
    return accuracy

In [ ]:
epochs = 200
train_losses = []
val_accuracies = []
best_val_acc = 0.0
patience = 30
counter = 0

for epoch in range(1, epochs + 1):
    loss = train()
    train_losses.append(loss)

    val_acc = evaluate(data.val_mask)
    val_accuracies.append(val_acc)
    

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        counter = 0  # Reset counter kalo ada improvement
    else:
        counter += 1  # Increment kalau tidak ada improvement
    
    # kehabisan kesabaran
    if counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break
    
    # progress setiap 20 epoch
    if epoch % 20 == 0:
        train_acc = evaluate(data.train_mask)
        test_acc = evaluate(data.test_mask)
        print(f'Epoch {epoch:3d} | Loss: {loss:.4f} | Train: {train_acc:.4f} | Val: {val_acc:.4f} | Test: {test_acc:.4f}')

# EVALUATE ON FULL GRAPH

In [ ]:
train_acc = evaluate(data.train_mask)
val_acc = evaluate(data.val_mask)
test_acc = evaluate(data.test_mask)

In [ ]:
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Val Accuracy:   {val_acc:.4f}")
print(f"Test Accuracy:  {test_acc:.4f}")

# TRAINING DYNAMICS PLOT

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
# Loss plot
axes[0].plot(train_losses, label='Training Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training Loss over Epochs', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy plot
axes[1].plot(val_accuracies, label='Validation Accuracy', linewidth=2, color='orange')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Validation Accuracy over Epochs', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


# PREDICTION EXAMPLE

In [ ]:
model.eval()
with torch.no_grad():
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)

In [ ]:
# Show 10 random test nodes
test_indices = data.test_mask.nonzero(as_tuple=True)[0]
sample_indices = test_indices[torch.randperm(len(test_indices))[:10]]

In [ ]:
print(f"{'Node ID':<10} {'True Label':<12} {'Predicted':<12} {'Correct?':<10}")
print("-" * 50)
for idx in sample_indices:
    true_label = data.y[idx].item()
    pred_label = pred[idx].item()
    correct = "correct" if true_label == pred_label else "incorrect"
    print(f"{idx.item():<10} {true_label:<12} {pred_label:<12} {correct:<10}")

In [ ]:
model